# Oeffikator

## Setup
We start off with global varibale and function definition and important imports.

In [1]:
import asyncio
import datetime
import geopy.distance
import json
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import multiprocessing as mp
import nest_asyncio
import numpy as np
import os
import pandas as pd
import random 
import requests
import time

from apis.OeffiAPI import OeffiAPI
from apis.BVGRestAPI import BVGRestAPI
from concurrent.futures import ThreadPoolExecutor
from TimeTransformer import TimeTransformer
from scipy.spatial import Delaunay

In [2]:
DAY = datetime.datetime.today() + datetime.timedelta(days=1)
while DAY.weekday()!= 0:
    DAY+=datetime.timedelta(1)
cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),
         'blue':  ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))}
CMAP = mcolors.LinearSegmentedColormap('my_colormap', cdict, 100)

In [3]:
def compute_distance(x1,y1,x2,y2):
    return geopy.distance.distance((x1,y1), (x2,y2)).km

def to_BVG_sdt(x : float) -> int:
    return int(x*10E5)

async def start_async_process(apis, origin, tri_centers, starting_date):
    amount_of_workers = len(tri_centers)
    journeys = []
    with ThreadPoolExecutor(max_workers=amount_of_workers) as executor:
        loop = asyncio.get_event_loop()
        responses = [
            loop.run_in_executor(
                executor,
                apis[i%len(apis)].get_journey,
                *(origin,{"longitude": str(tri_centers[i][0]), "latitude": str(tri_centers[i][1]), "address": "placeholder"}, starting_date)
            )
            for i in range(amount_of_workers)
        ]
        for journey in await asyncio.gather(*responses):
            journeys.append(journey)
    return journeys

## Crawl Data

Let's define some parameter first.

In [4]:
api = BVGRestAPI()
api2 = OeffiAPI()
apis = [api, api2]
query = "maxstr 5, wedding"
start = api.query_location(query)
print("We gonne use following address: " + start["address"])

We gonne use following address: 13347 Berlin-Wedding, Maxstr. 5


In [5]:
file_name = str(to_BVG_sdt(start["longitude"])) + "_" + str(to_BVG_sdt(start["latitude"])) + ".csv"
# in format left, right, bottom, top
# or xmin, xmax, ymin, ymax
bounding_box = (13.2756, 13.4892, 52.4677, 52.5532)
step_size_x = 0.01
step_size_y = step_size_x / 2

length = len(np.arange(bounding_box[0],bounding_box[1],step_size_x)) * len(np.arange(bounding_box[2],bounding_box[3],step_size_y))
print("Amount of requests: ",length)
print("Estimated time: " + str(int(length/38)) + "min")

Amount of requests:  396
Estimated time: 10min


## Linear Computing

And now let's get the points.

In [ ]:
i = 0
destination_i = 0
df = pd.DataFrame(columns = ['X', 'Y', 'Time'])
start_time = datetime.datetime.now()
for x in np.arange(bounding_box[0],bounding_box[1],step_size_x):
    for y in np.arange(bounding_box[2],bounding_box[3],step_size_y):
        destination_i+=1
        destination = {"longitude": str(x), "latitude": str(y), "address": "placeholder"}
        print(destination_i/length*100,"% Fortschritt, es fehlen",length - destination_i)
        print("Current Destination: " + str(x) + ", " + str(y))
        try:
            #time.sleep(60/100)
            journey = api.get_journey(start,destination,DAY)
            for stop in journey["stopovers"]:
                i+=1
                df.loc[i] = [stop["longitude"], stop["latitude"], stop["time"]]
            i+=1
            df.loc[i] = [journey["destination"]["longitude"], journey["destination"]["latitude"], journey["arrivalTime"]]
        except (ValueError, KeyError):
            df.loc[i] = [destination["longitude"], destination["latitude"], "error"]
            continue

        if datetime.datetime.now() - start_time > datetime.timedelta(seconds=32):
            df = df.drop_duplicates()
            df.to_csv("results/"+file_name, index=True, header=False)
            start_time = datetime.datetime.now()

df = df.drop_duplicates()
df.to_csv("results/"+file_name, index=True, header=False)

## Parallel computing

And one alternative method of computing the map.

In [6]:
parallel_threads = 8

In [7]:
def get_area(points):
    first = np.multiply(points[:,0,0], np.subtract(points[:,1,1],points[:,2,1]))
    second = np.multiply(points[:,1,0], np.subtract(points[:,2,1],points[:,0,1]))
    third = np.multiply(points[:,2,0], np.subtract(points[:,0,1],points[:,1,1]))
    return 0.5*np.add(np.add(first,second),third)

i = 0
destination_i = 0
df = pd.DataFrame(columns = ['longitude', 'latitude', 'Time'])
start_time = datetime.datetime.now()
for x in np.linspace(bounding_box[0], bounding_box[1], 3):
    for y in np.linspace(bounding_box[2], bounding_box[3], 3):
        print(f"So far {i} points were generated of which {destination_i} are destinations.")
        i += 1
        destination_i += 1
        destination = {"longitude": str(x), "latitude": str(y), "address": "placeholder"}
        journey = api.get_journey(start,destination,DAY)
        df.loc[i] = [journey["destination"]["longitude"], journey["destination"]["latitude"], journey["arrivalTime"]]
time_taken_total = datetime.datetime.now() - start_time
print("This took us:")
print(time_taken_total)

So far 0 points were generated of which 0 are destinations.
So far 1 points were generated of which 1 are destinations.
So far 2 points were generated of which 2 are destinations.
So far 3 points were generated of which 3 are destinations.
So far 4 points were generated of which 4 are destinations.
So far 5 points were generated of which 5 are destinations.
So far 6 points were generated of which 6 are destinations.
So far 7 points were generated of which 7 are destinations.
So far 8 points were generated of which 8 are destinations.
This took us:
0:00:14.001347


Running the requests asynchonously which speeds up the requests per minute up to > 200.

In [9]:
nest_asyncio.apply()

start_time = datetime.datetime.now()
round_time = datetime.datetime.now()
while datetime.datetime.now() - start_time < datetime.timedelta(minutes=2):
#while destination_i < 48:
    available_apis = [api for api in apis if not api.has_reached_request_limit()]
    for api in available_apis:
        print(api,len(api.past_requests))
    if available_apis:
        print(f"So far {i} points were generated of which {destination_i} are destinations.")



        points = np.array(list(zip(df["longitude"], df["latitude"])))
        tri = Delaunay(points)
        triangle = points[tri.simplices][np.argmax(get_area(points[tri.simplices]))]
        areas = get_area(points[tri.simplices])
        # get the 5 largest areas
        tri_centers = (np.mean(points[tri.simplices][np.argpartition(areas, -parallel_threads)[-parallel_threads:]], 1))

        #Run asychnronous requests
        loop = asyncio.get_event_loop()
        journeys = asyncio.ensure_future(start_async_process(available_apis, start, tri_centers, DAY))
        loop.run_until_complete(journeys)
        
        for journey in journeys.result():
            destination_i += 1
            if journey["stopovers"] != None:
                for stop in journey["stopovers"]:
                    i+=1
                    df.loc[i] = [stop["longitude"], stop["latitude"], stop["time"]]
            i+=1
            df.loc[i] = [float(journey["destination"]["longitude"]), float(journey["destination"]["latitude"]), journey["arrivalTime"]]
    else: 
        print("All apis have reached there request threshold. Sleeping ...")
        time.sleep(2)

    if datetime.datetime.now() - round_time > datetime.timedelta(seconds=60):
        df = df.drop_duplicates()
        df.to_csv("results/new_"+file_name, index=True, header=False)
        round_time = datetime.datetime.now()

df = df.drop_duplicates()
df.to_csv("results/new_"+file_name, index=True, header=False)

time_taken_total = datetime.datetime.now() - start_time
print("This took us:")
print(time_taken_total)

<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 9
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 20
So far 282 points were generated of which 49 are destinations.
<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 12
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 20
So far 328 points were generated of which 54 are destinations.
<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 9
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 19
So far 366 points were generated of which 59 are destinations.
<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 11
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 21
So far 399 points were generated of which 64 are destinations.
<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 12
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 24
So far 419 points were generated of which 69 are destinations.
<apis.BVGRestAPI.BVGRestAPI object at 0x7ff5c6eaeeb0> 15
<apis.OeffiAPI.OeffiAPI object at 0x7ff5c6eaed60> 30
So far 444 points were gene

In [ ]:
lst = [0,0,0,0,0,1,1,1,1]
while lst[0] < 1:
    lst.pop(0)
lst

(Different Method to generate directions https://stackoverflow.com/questions/57539749/find-out-centre-of-the-most-dense-region-in-a-scatter-plot )

In [ ]:
# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
# k = kde.gaussian_kde(data.T)
# xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
# zi = k(np.vstack([xi.flatten(), yi.flatten()]))

Time to get the data.

## Plotting

Let's read data first.

In [ ]:
query = "maxstr.5, 13347"

In [ ]:
start = api.query_location(query)
file_name_plottting = str(to_BVG_sdt(start["longitude"])) + "_" + str(to_BVG_sdt(start["latitude"])) + ".csv"
df = pd.read_csv("results/new_" + file_name_plottting, sep=",", index_col = 0, names=["X","Y","Time"],  na_values="None", dtype = {"X": np.float32, "Y": np.float32, "Time":str})

And then transform it with some custom pipelines.

In [ ]:
time_transformer = TimeTransformer()
dfn = time_transformer.transform(df)

It's time to plot:

In [ ]:
dfn.head()
dfn.loc[393]

In [ ]:
dfn = dfn[dfn["Time"]<200]

In [ ]:
#get the start location coordinates from the file name
start_location = tuple([int(coordinate)*10E-7 for coordinate in file_name_plottting[:-4].split("_")])
# get the bounding box from the location
bounding_box_locations = (min(dfn["X"]), max(dfn["X"]), min(dfn["Y"]), max(dfn["Y"]))
# sets hard coded for the map "map_berlin_A.png"
bounding_box_map = (13.272, 13.491, 52.456, 52.563)
map = plt.imread('maps/map_berlin_A.png')
# define the amount of color levels should be there
levels = np.linspace(np.min(dfn["Time"]), np.max(dfn["Time"]), 32)

fig, ax = plt.subplots(figsize = (18,18))
ax.set_xlim(bounding_box[0],bounding_box[1])
ax.set_ylim(bounding_box[2],bounding_box[3])
ax.tricontourf(dfn["X"], dfn["Y"], dfn["Time"], levels=levels, alpha= 0.5, cmap = CMAP, antialiased=True)
#Displaying destination locations
ax.scatter(dfn["X"], dfn["Y"], alpha=0.5, color = "brown")

# "aspect=1.65" as a magic number
ax.imshow(map, extent = bounding_box_map, aspect = 1.65)
# plot the starting position
ax.plot(start_location[0], start_location[1], marker="*", markersize=20, color="tab:orange")

In [ ]:

plt.savefig("results/images/map_berlin_sWedding.png")

## Creating a map
This code is copied from following website: http://bigmap.osmz.ru/

In [ ]:
#!/usr/bin/env python
# Generated by BigMap 2. Permalink: http://bigmap.osmz.ru/bigmap.php?xmin=17592&xmax=17611&ymin=10740&ymax=10755&zoom=15&scale=256&tiles=osm-de

import io, urllib.request, datetime, time, re, random
from PIL import Image, ImageDraw
# ^^^^^^ install "python-pillow" package | pip install Pillow | easy_install Pillow

(zoom, xmin, ymin, xmax, ymax) = (15, 17592, 10740, 17611, 10755)
layers = ["https://{abc}.tile.openstreetmap.de/tiles/osmde/!z/!x/!y.png"]
attribution = 'Map data (c) OpenStreetMap, Tiles (c) OSM DE'
xsize = xmax - xmin + 1
ysize = ymax - ymin + 1
tilesize = 256

resultImage = Image.new("RGBA", (xsize * tilesize, ysize * tilesize), (0,0,0,0))
counter = 0
for x in range(xmin, xmax+1):
	for y in range(ymin, ymax+1):
		for layer in layers:
			url = layer.replace("!x", str(x)).replace("!y", str(y)).replace("!z", str(zoom))
			match = re.search("{([a-z0-9]+)}", url)
			if match:
				url = url.replace(match.group(0), random.choice(match.group(1)))
			print(url, "... ");
			try:
				req = urllib.request.Request(url, headers={'User-Agent': 'BigMap/2.0'})
				tile = urllib.request.urlopen(req).read()
			except Exception as e:
				print("Error", e)
				continue;
			image = Image.open(io.BytesIO(tile))
			resultImage.paste(image, ((x-xmin)*tilesize, (y-ymin)*tilesize), image.convert("RGBA"))
			counter += 1
			if counter == 10:
				time.sleep(2);
				counter = 0

draw = ImageDraw.Draw(resultImage)
draw.text((5, ysize*tilesize-15), attribution, (0,0,0))
del draw

now = datetime.datetime.now()
outputFileName = "maps/map_Berlin_A_high.png"# % (zoom, now.year % 100, now.month, now.day, now.hour, now.minute)
resultImage.save(outputFileName)

## Previous code

In [ ]:
import json
import requests
url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"LocGeoPos","cfg":{"polyEnc":"GPA"},"req":{"ring":{"cCrd":{"x":%d,"y":%d},"maxDist":20000},"getStops":true,"getPOIs":true,"maxLoc":1}}],"formatted":false}'
s1 = s1 % (key,13430400,52536099)
data = json.loads(s1)
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
r = requests.post(url, data=json.dumps(data), headers=headers)
#print(r.text)
resp = json.loads(r.text)
resp["svcResL"][1]['res']['locL'][0]['lid']

In [ ]:
rp = """{"ver":"1.18","ext":"BVG.1","lang":"eng","id":"tm2ktgxukkk3y2wg","err":"OK","svcResL":[{"meth":"ServerInfo","err":"OK","res":{"common":{"locL":[],"prodL":[],"polyL":[],"layerL":[{"id":"standard","name":"standard","index":0,"annoCnt":0}],"crdSysL":[{"id":"standard","index":0,"type":"WGS84"}],"opL":[],"remL":[],"icoL":[]},"sD":"20220202","sT":"114242","enc":"ISO-8859-1","planrtTS":"1643798528"}},{"meth":"LocGeoPos","err":"OK","res":{"common":{"locL":[],"prodL":[{"pid":"L::2::Tram::B3041857784::Tram_3041857784_M10::*","name":"M10","nameS":"M10","icoX":0,"cls":4,"prodCtx":{"name":"M10","line":"M10","lineId":"M10","catOut":"Tram    ","catOutS":"T","catOutL":"Tram    "},"himIdL":["HIM_FREETEXT_75748","HIM_FREETEXT_52249"]}],"polyL":[],"layerL":[{"id":"standard","name":"standard","index":0,"annoCnt":0}],"crdSysL":[{"id":"standard","index":0,"type":"WGS84"}],"opL":[],"remL":[],"icoL":[{"res":"prod_gen"},{"res":"POI"}]},"locL":[{"lid":"A=4@O=Berlin, Theater unterm Dach@X=13430936@Y=52537482@u=158@U=104@L=900980701@","type":"P","name":"Berlin, Theater unterm Dach","icoX":1,"extId":"900980701","state":"F","crd":{"x":13430936,"y":52537482,"layerX":0,"crdSysX":0},"dist":158,"dur":309}]}}]}
"""
resp = json.loads(rp)
a = resp["svcResL"][1]['res']['locL'][0]['extId']
b = resp["svcResL"][1]['res']['locL'][0]['type']
print(a,b)

resp

In [ ]:
import geopy.distance

F = 10E-7
def distance(x,y,x2,y2):
    coords_1 = (y*F, x*F)
    coords_2 = (y2*F, x2*F)
    return geopy.distance.distance(coords_1, coords_2).km


F = 10E-7
import json
import requests
url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"LocGeoPos","cfg":{"polyEnc":"GPA"},"req":{"ring":{"cCrd":{"x":%d,"y":%d},"maxDist":20000},"getStops":true,"getPOIs":true,"maxLoc":1}}],"formatted":false}'
s1 = s1 % (key)
teststops = []
for x in range(13440000,13490000,2000):
    print(x)
    y= 52536099
    
    s1_con = s1 % (x,y)
    print(distance(x*F,y*F,13430400*F,52536099*F))
    data = json.loads(s1_con)
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post(url, data=json.dumps(data), headers=headers)
    #print(r.text)
    resp = json.loads(r.text)
    stop = resp["svcResL"][1]['res']['locL'][0]['lid']
    teststops.append(stop)

In [ ]:
teststops

In [ ]:
#messung wie weit die generierte station und die urpsprünglichen koordinaten auseinander liegen
#ist 350m wenig genug?
distance(13355768,52635339,13350744,52636099)

In [ ]:
distance(13355000*F,52635000*F,13357000*F,52635000*F)*1000

In [ ]:
import json
import requests
url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"TripSearch","cfg":{"polyEnc":"GPA"},"req":{"depLocL":[{"type":"P","lid":"A=4@O=Berlin, Prenzlkasper@X=13424679@Y=52536107@u=109@U=104@L=900980734@"}],"arrLocL":[{"type":"S","extId":"900200507"}],"outDate":"20220202","outTime":"120000","outFrwd":true,"gisFltrL":[{"mode":"FB","profile":{"type":"F","linDistRouting":false,"maxdist":2000},"type":"M","meta":"foot_speed_normal"}],"getPolyline":true,"getPasslist":true,"getConGroups":false,"getIST":false,"getEco":false,"extChgTime":-1}}],"formatted":false}'
s1 = s1 % (key)
#s1 = s1 % (13430400,52536099)
data = json.loads(s1)
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
r = requests.post(url, data=json.dumps(data), headers=headers)
#print(r.text)
resp = json.loads(r.text)
for x in range(len(resp['svcResL'][1]['res']['outConL'])):
    print(resp['svcResL'][1]['res']['outConL'][x]['arr']['aTimeS'])

    print("")

In [ ]:
length = len(range(13368007,13390623,2000)) * len(range(52518214,52523802,2000))
length2 = len(range(13368007,13390623,2000)) * len(range(52518214,52543802,2000))
print(length)
print(length2)

In [ ]:
import geopy.distance

import json
import requests
#test setup: x,y
#punkt 1:52523802, 13368007
#punkt 2:52518214, 13390623
#startpunkt beier
#punkt 3:52541350,  13366540
# x sind die 13er zahlen
F = 10E-7
def distance(x,y,x2,y2):
    coords_1 = (y*F, x*F)
    coords_2 = (y2*F, x2*F)
    return geopy.distance.distance(coords_1, coords_2).km



quit()
atimes = []
i =0

#links oben moabit 52.53746136684121, 13.31176629214652
#rechts unten moabit 52.51996891399787, 13.366440440707555
#xxxxxxxxxx
X1=13311766
Y1=52519960


X2=13366440
Y2=52537461

print("X distance gps",X2-X1)
print("Y distance gps", Y2-Y1)

print("X distance km",distance(X1,Y1,X2,Y1))
print("Y distance km",distance(X2,Y1,X2,Y2))

length = len(range(X1,X2,2000)) * len(range(Y1,Y2,2000))
print("anzahl berechnungen:",length)

start_lid = get_lid(13366540,52541350)
print(start_lid)


for x in range(13368007,13390623,2000):
    for y in range(52518214,52543802,2000):
        try:
            print(x,y)
            i+=1
            print(i/length*100,"% Fortschritt, es fehlen ",length - i)

            dest = get_dest(x,y)
            #print(get_lid(x,y))
            print("destination paramteres:",dest)
            time = get_traveltime(start_lid,dest)
            print(time)
            f = open("results3.txt","a+")
            f.write(str(x)+","+str(y)+","+str(i)+","+str(time)+"\n")
            f.close()

            #print(distance(13366540,52541350,x,y))
            atimes.append((x,y,time))
        except ValueError:
            with open("results3.txt","a+") as f:
                f.write("error"+str(i)+",x="+str(x)+", "+str(y)+"\n")

            continue
            
            
            

In [ ]:
def get_lid(x,y):
    url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
    s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"LocGeoPos","cfg":{"polyEnc":"GPA"},"req":{"ring":{"cCrd":{"x":%d,"y":%d},"maxDist":20000},"getStops":true,"getPOIs":true,"maxLoc":1}}],"formatted":false}'
    s1 = s1 % (key,x,y)
    data = json.loads(s1)
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post(url, data=json.dumps(data), headers=headers)
    #print(r.text)
    resp = json.loads(r.text)
    return resp["svcResL"][1]['res']['locL'][0]['lid']

def get_dest(x,y):
    url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
    s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"LocGeoPos","cfg":{"polyEnc":"GPA"},"req":{"ring":{"cCrd":{"x":%d,"y":%d},"maxDist":20000},"getStops":true,"getPOIs":true,"maxLoc":1}}],"formatted":false}'
    s1 = s1 % (key,x,y)
    data = json.loads(s1)
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post(url, data=json.dumps(data), headers=headers)
    #print(r.text)
    resp = json.loads(r.text)
    e = resp["svcResL"][1]['res']['locL'][0]['extId']
    t = resp["svcResL"][1]['res']['locL'][0]['type']
    return (e,t)

def get_traveltime(start_lid,dest):
    extId = dest[0]
    dest_type = dest[1]
    url = "http://bvg-apps-ext.hafas.de/bin/mgate.exe/mgate.exe"
    s1 = '{"auth":{"aid":"%s","type":"AID"},"client":{"id":"BVG","type":"AND"},"ext":"BVG.1","ver":"1.18","lang":"eng","svcReqL":[{"meth":"ServerInfo","req":{"getServerDateTime":true,"getTimeTablePeriod":false}},{"meth":"TripSearch","cfg":{"polyEnc":"GPA"},"req":{"depLocL":[{"type":"P","lid":"%s"}],"arrLocL":[{"type":"%s","extId":"%s"}],"outDate":"20220202","outTime":"120000","outFrwd":true,"gisFltrL":[{"mode":"FB","profile":{"type":"F","linDistRouting":false,"maxdist":2000},"type":"M","meta":"foot_speed_normal"}],"getPolyline":true,"getPasslist":true,"getConGroups":false,"getIST":false,"getEco":false,"extChgTime":-1}}],"formatted":false}'
    #s1: 1. lid, 2. type, 3. extId
    s1 = s1 % (key,start_lid,dest_type, extId)
    data = json.loads(s1)
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post(url, data=json.dumps(data), headers=headers)
    #print(r.text)
    resp = json.loads(r.text)
    #for x in range(len(resp['svcResL'][1]['res']['outConL'])):
    try:
        aTime = resp['svcResL'][1]['res']['outConL'][0]['arr']['aTimeS']
        print(resp['svcResL'][1]['res']['outConL'][0]['arr'])
    except:
        print("error!")
        print(resp)
        return None
    if aTime == None:
        print(resp['svcResL'][1]['res']['outConL'][0]['arr'])

        return None
    return int(aTime)


In [ ]:
with open("results2.txt") as r:
        X = []
        Y = []
        L = []
        for l in r:
            if "error" in l:
                continue
            x = l.split(",")[0]
            y = l.split(",")[1]
            time = l.split(",")[3]
            hour = time[0:2]
            minutes = time[2:4]
            length = int(minutes) + (int(hour) - 12)*60
            #print(length)
            X.append(int(x))
            Y.append(int(y))
            L.append(length)

In [ ]:
len(L)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from sys import argv
import matplotlib.pyplot as plt
ax = plt.axes(projection='3d')
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
import numpy as np
#fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
#ax.scatter3D(np.array(X),np.array(Y),np.array(length))

#ax.scatter3D(X,Y,L,c=L)

fig = plt.figure()
ax = Axes3D(fig)
ax.view_init(azim=180)

surf = ax.plot_trisurf(X, Y, L, cmap=cm.jet, linewidth=0.1)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig('teste.pdf')
plt.show()



print("Making animation")


### Visualisation of Triangulization

The underlying idea is the computation of traingles and identifying the largest area. This code block visualizes the working principle.

In [ ]:


start_location = [start["longitude"], start["latitude"]]
# get the bounding box from the location
#bounding_box_locations = (min(dfn["longitude"]), max(dfn["longitude"]), min(dfn["latitude"]), max(dfn["latitude"]))
# sets hard coded for the map "map_berlin_A.png"
bounding_box_map = (13.272, 13.491, 52.456, 52.563)

fig, ax = plt.subplots(figsize = (12,12))

def plotting():
    map = plt.imread('maps/map_berlin_A-min.png')
    # define the amount of color levels should be there
    levels = np.linspace(0, 90, 32)

    ax.set_xlim(bounding_box_map[0],bounding_box_map[1])
    ax.set_ylim(bounding_box_map[2],bounding_box_map[3])
    #ax.tricontourf(dfn["longitude"], dfn["latitude"], dfn["Time"], levels=levels, alpha= 0.5, cmap = CMAP, antialiased=True)
    #Displaying destination locations
    #ax.scatter(dfn["longitude"], dfn["latitude"], alpha=0.5, color = "brown")

    # "aspect=1.65" as a magic number
    ax.imshow(map, extent = bounding_box_map, aspect = 1.65)
    # plot the starting position
    ax.plot(start_location[0], start_location[1], marker="*", markersize=20, color="tab:orange")
plotting()
display(fig)
clear_output(wait=True)



start_time = datetime.datetime.now()
round_time = start_time
while datetime.datetime.now() - start_time < datetime.timedelta(seconds=100):

        points = np.array(list(zip(df["longitude"], df["latitude"])))
        tri = Delaunay(points)
        triangle = points[tri.simplices][np.argmax(get_area(points[tri.simplices]))]
        triangle = np.vstack((triangle, triangle[0,:]))
        ax.plot(triangle[:,0], triangle[:,1])

        display(fig)
        clear_output(wait=True)
        time.sleep(0.5)



        point  = np.mean(points[tri.simplices][np.argmax(get_area(points[tri.simplices]))],0)
        x = point[0]
        y = point[1]
        ax.plot(x, y, marker="o", markersize=20, color="yellow")

        display(fig)
        clear_output(wait=True)
        #time.sleep(1)


        destination = {"longitude": str(x), "latitude": str(y), "address": "placeholder"}
        try:
            journey = api.get_journey(start,destination,DAY)
            for stop in journey["stopovers"]:
               i+=1
               df.loc[i] = [stop["longitude"], stop["latitude"], stop["time"]]
            i+=1
            df.loc[i] = [journey["destination"]["longitude"], journey["destination"]["latitude"], journey["arrivalTime"]]
        except (ValueError, KeyError):
            df.loc[i] = [destination["longitude"], destination["latitude"], "error"]
            continue

        dfn = df.copy()
        dfn = time_transformer.transform(dfn)
        asd = ax.remove()

        fig, ax = plt.subplots(figsize = (12,12))
        plotting()
        ax.scatter(dfn["longitude"], dfn["latitude"], alpha=1, color = "brown")


        display(fig)
        clear_output(wait=True)
        time.sleep(0.5)

